![banner_etl](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_automatizacion.gif?raw=true)

**<mark style="background:#2bfe9c">Script de automatización</mark> proviene del Notebook de Github <mark>ETL_metadata_GM</mark> y <mark>ETL_estados_GM</mark>**

#### Tenemos una estrategia la cual le daremos seguimiento, aunque tenemos un plan alterno, este Script se conecta directamente al directorio de `Google Drive` proporcionado por el cliente, aunque nos informa que es data estática si llegase el caso de actualizarse, nuestros datos se actualizarán de forma paralela, en conclusión `Google Drive` entra a ser parte de nuestro `DataLake`

Instalamos librerías para la conexión de la API de google drive

In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib # Librería de Google para API

StatementMeta(, , , SessionStarting, )

In [12]:
# Librerías necesarias para este notebook de automatización
import pandas as pd
import  numpy as np
import os
import json
import tempfile
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
import io
from googleapiclient.errors import HttpError

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 14, Finished, Available)

#### Función `list_files_in_folder()` crea el servicio de la API google con los parámetros y filtros para listar el folder solicitado; en este caso donde se encuentran los datasets de `Google Maps`, es necesario usar `supportsAllDrives=True, includeItemsFromAllDrives=True` para listar una carpeta compartida.

In [5]:
def list_files_in_folder(service, folder_id):
    results = service.files().list(supportsAllDrives=True, includeItemsFromAllDrives=True,
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)").execute()
    return results.get("files", [])

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 7, Finished, Available)

#### Función para Filtrar y leer todos los datasets `.JSON` que se encuentran dentro de una carpeta específica, existen dos carpetas posibles `metadata-sitios` y `reviews-estados`

In [6]:
def list_all_json_files(service, folder_id, folder_name):
    json_files = []
    stack = [folder_id]

    while stack:
        current_folder_id = stack.pop()
        files = list_files_in_folder(service, current_folder_id)
        
        for file in files:
            if file["mimeType"] == "application/json":
                json_files.append(file)
            elif file["mimeType"] == "application/vnd.google-apps.folder":
                # Solo agregamos carpetas a la pila si son subcarpetas de la carpeta "metadata-sitios"
                if folder_name in file["name"]:
                    stack.append(file["id"])

    return json_files

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 8, Finished, Available)

#### Esta función hace la lectura del archivo específico en este caso `.JSON`, la convierte en un dataframe de pandas, al tener una estructura complicada para la librería `json` se procede a generar un archivo temporal con la librería `tempfile` y leer de inmediato el JSON con Pandas.

In [9]:
def read_json_file(service, file_id):
    content = service.files().get_media(fileId=file_id).execute()
    # Crear un archivo temporal y escribir el contenido
    with tempfile.NamedTemporaryFile(delete=False, suffix=".json") as temp_file:
        temp_file.write(content)
        temp_file.seek(0)
        return pd.read_json(temp_file, lines=True)

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 11, Finished, Available)

In [ ]:
def list_subfolders_in_folder(service, folder_id):
    results = service.files().list(
        supportsAllDrives=True,
        includeItemsFromAllDrives=True,
        q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'",
        fields="files(id, name)").execute()
    return results.get("files", [])

In [10]:
# Ruta al archivo JSON de credenciales de servicio
credentials_path = "/lakehouse/default/Files/proyecto-server-18-85d41d78e0f8.json"
# Ruta Scope necesaria para el funcionamiento de la API
scope = ['https://www.googleapis.com/auth/drive']

# Crea una instancia de autenticación usando las credenciales de servicio
credentials = service_account.Credentials.from_service_account_file(
    filename=credentials_path,
    scopes=scope)

# Crea una instancia del servicio de Google Drive
service = build("drive", "v3", credentials=credentials)

# Directorio principal el cual es suministrado por el cliente
directory_id = "1Wf7YkxA0aHI3GpoHc9Nh8_scf5BbD4DA"

folder_names = ["metadata-sitios", "reviews-estados"]

dfs = {}  # Un diccionario para almacenar los DataFrames

for folder_name in folder_names:
    if folder_name == "reviews-estados":
        # Obtener las subcarpetas dentro de la carpeta "reviews-estados"
        reviews_folder_id = list_subfolders_in_folder(service, directory_id, "reviews-estados")

        for reviews_subfolder in reviews_folder_id:
            reviews_subfolder_id = reviews_subfolder["id"]
            state_name = reviews_subfolder["name"].replace("review-", "")  # Nombre del estado
            json_files = list_all_json_files(service, reviews_subfolder_id)

            df = pd.concat([read_json_file(service, json_file["id"]).assign(estado=state_name) for json_file in json_files], ignore_index=True)
            print(f"Concatenando DataFrames para {state_name}")

        dfs[folder_name] = df

    else:
        json_files = list_all_json_files(service, directory_id, folder_name)
        df = pd.concat([read_json_file(service, json_file["id"]) for json_file in json_files], ignore_index=True)
        print(f"Concatenando DataFrames para {folder_name}")
        dfs[folder_name] = df

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 12, Finished, Available)

Se ha concatenado un DataFrame para el archivo 11.json
Se ha concatenado un DataFrame para el archivo 10.json
Se ha concatenado un DataFrame para el archivo 9.json
Se ha concatenado un DataFrame para el archivo 8.json
Se ha concatenado un DataFrame para el archivo 7.json
Se ha concatenado un DataFrame para el archivo 6.json
Se ha concatenado un DataFrame para el archivo 5.json
Se ha concatenado un DataFrame para el archivo 4.json
Se ha concatenado un DataFrame para el archivo 3.json
Se ha concatenado un DataFrame para el archivo 2.json
Se ha concatenado un DataFrame para el archivo 1.json


In [ ]:
df_metadata = dfs["metadata-sitios"].copy()
df_reviews = dfs["reviews-estados"].copy()

In [13]:
df_metadata['category_hotel'] = np.nan

#filtro del dataframe para el rubro 'hotel' (columna 'category')
for fila in range(len(df_metadata['category'])):
    if type(df_metadata['category'][fila]) == list:
        if 'Hotel' in df_metadata['category'][fila]:
            df_metadata.loc[fila, 'category_hotel'] = 'Hotel'
df_metadata = df_metadata[df_metadata['category_hotel'] == 'Hotel']

StatementMeta(, 28e0cdc3-bee9-4662-8bcb-1518645bb0a4, 15, Finished, Available)

In [1]:
# Directorio donde deseas guardar los archivos descargados
directorio_destino = "/lakehouse/default/Files/data/original/Google/"

StatementMeta(, , , Cancelled, )

In [ ]:
# Generar el Csv filtrado por Hoteles
df_metadata.to_csv(directorio_destino + "df_metadata.csv", index=False)

In [ ]:
df_reviews = pd.merge(df_metadata, df_reviews, on='gmap_id', how='inner')

In [ ]:
# Generar el Csv de reviews filtrado por Hoteles
df_reviews.to_csv(directorio_destino + "df_reviews.csv", index=False)